In [ ]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from scipy.interpolate import RectBivariateSpline
from remeshing import get_area
from scipy.spatial import cKDTree

import os
from datetime import datetime, timedelta
import glob
from scipy.ndimage import gaussian_filter
from tqdm.notebook import tqdm

from utils import get_mesh_files, fill_gaps

%matplotlib inline

In [ ]:
def load_sic_data(ifile, mesh_src_file):
    with np.load(ifile) as d:
        sic_unc = d['sic_unc']
    with np.load(mesh_src_file) as d:
        x = d['x']
        y = d['y']
        t = d['t']
        src2dst = d['src2dst']
        weights = d['weights']
    return sic_unc, x, y, t, src2dst, weights


In [ ]:
# filter size (Gaissian filter was 5, but weights are smaller for larger distances)
size = 3

# should be 2210 to cover 6 years
n_steps = 2210 

In [ ]:
sia_dir = '/data2/Anton/sia/cdr_1991_2023'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
sid_dir = '/data2/Anton/sia_sid_cdr_postproc'
ifiles = sorted(glob.glob(f'{sid_dir}/*/*npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']
mask_f = gaussian_filter(mask.astype(float), 1, truncate=1)

In [ ]:
start_indices = [i for i,j in enumerate(idates) if j.year > 2015 and j.month == 9 and j.day == 5]
start_indices

In [ ]:
for start_idx in start_indices:
    stop_idx = min(start_idx + n_steps, len(idates))
    unc_sic_sum = None
    start_date = idates[start_idx]
    odir = f'{unc_dir}/{start_date.year}'
    os.makedirs(odir, exist_ok=True)
    for i in tqdm(range(start_idx, stop_idx), total=n_steps):
        ifile = ifiles[i]
        idate = idates[i]
        ofile = f'{odir}/unc_sic_{start_date.strftime("%Y%m%d")}_{idate.strftime("%Y%m%d")}.npz'
        if os.path.exists(ofile):
            with np.load(ofile) as d:
                unc_sic_sum = d['unc_sic']
                sic_min = d['sic_min']
            continue

        mesh_file, mesh_dst_file = get_mesh_files(idate, mesh_dir, mesh_init_file)
        unc_sic_grd, x, y, t, src2dst, weights = load_sic_data(ifile, mesh_file)
        unc_sic_fil = fill_gaps(unc_sic_grd, np.isnan(unc_sic_grd), distance=144)
        
        # interpolate SIC uncertainty to mesh
        unc_sic = RectBivariateSpline(xc, yc, unc_sic_fil[::-1], kx=1, ky=1)(y[t].mean(axis=1), x[t].mean(axis=1), grid=False)

        sic_file = mesh_file.replace('mesh', 'sic')
        c = np.load(sic_file)['c']

        if unc_sic_sum is None:
            # first time step
            unc_sic_sum = np.array(unc_sic)
            sic_min = np.array(c)
        else:
            # advect unc_sic_sum (from previous time step)
            unc_sic_sum_pro = np.zeros(src2dst[:,1].max()+1)
            np.add.at(unc_sic_sum_pro, src2dst[:,1], unc_sic_sum[src2dst[:,0]] * weights)
            
            # advect sic_min from previous time step
            sic_min_pro = np.zeros(src2dst[:,1].max()+1)
            np.add.at(sic_min_pro, src2dst[:,1], sic_min[src2dst[:,0]] * weights)

            # keep uncertainty of the minimal value of SIC
            min_sic_ids = c < sic_min_pro
            unc_sic_sum = unc_sic_sum_pro
            unc_sic_sum[min_sic_ids] = unc_sic[min_sic_ids]
            
            # keep minimal ice concentration
            sic_min = np.min([sic_min_pro, c], axis=0)
        np.savez(ofile, unc_sid=unc_sic_sum, sic_min=sic_min)

    print(start_date)
    fig, axs = plt.subplots(1, 1, figsize=(6, 6))
    trp0 = axs.tripcolor(x, y, t, unc_sic_sum, cmap='jet')
    plt.colorbar(trp0, ax=axs)
    plt.show()            



In [ ]:
ofile